In [2]:
#Import libraries
import csv
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [3]:
crime = pd.read_csv("../../data/train.csv")

In [4]:
# Set the randomizer seed so results are the same each time.
np.random.seed(0)

#Recode weekday variable
weekday_mapping =  {'Monday': 1,
                    'Tuesday': 2,
                    'Wednesday': 3,
                    'Thursday': 4,
                    'Friday': 5,
                    'Saturday': 6,
                    'Sunday': 7}
crime['DayOfWeek']=crime['DayOfWeek'].map(weekday_mapping)

#Recode PdDistrict variable
enc = preprocessing.LabelEncoder()
district = enc.fit_transform(crime.PdDistrict)
crime['District'] = district

#Scale the coordinates variables
#stdsc = StandardScaler()
#crime['X_stdsc'] = stdsc.fit_transform(crime['X'])
#crime['Y_stdsc'] = stdsc.fit_transform(crime['Y'])


#Extract date and time information from 'Dates' variable
crime['Dates'] = pd.to_datetime(crime['Dates'])
crime['Year'] = crime['Dates'].dt.year
crime['Month'] = crime['Dates'].dt.month
crime['Day'] = crime['Dates'].dt.day
crime['Hour'] = crime['Dates'].dt.hour
crime['Minute'] = crime['Dates'].dt.minute


#Shuffle data
crime = crime.reindex(np.random.permutation(crime.index))

In [5]:
def baseline():
    #Only use the coordinates in this naive baseline model
    X = crime[['X','Y']]
    Y = crime.Category

    test_data, test_labels = X[710000:], Y[710000:]
    dev_data, dev_labels = X[700000:710000], Y[700000:710000]
    train_data, train_labels = X[:700000], Y[:700000]
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(train_data, train_labels)
    knn_prob = knn.predict_proba(test_data)
    
    test_labels_long = pd.get_dummies(test_labels)
    test_labels_long = pd.DataFrame.as_matrix(test_labels_long)
    logloss = log_loss(test_labels_long, knn_prob) 
    
    print ('Accuracy: ', knn.score(test_data,test_labels))
    print ('Log loss: ', logloss)    

baseline()

Accuracy:  0.249534362002
Log loss:  12.9395479919
